<a href="https://colab.research.google.com/github/yoseforaz0990/ML-templates/blob/main/deep_learning/AutoEncoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

| Step Name                                              | Description                                                                                                 |
|--------------------------------------------------------|-------------------------------------------------------------------------------------------------------------|
| Downloading the Dataset                               | This step involves downloading the MovieLens datasets (ml-100k and ml-1m) from the provided URLs and extracting them. |
| Importing the Libraries                               | The necessary libraries (NumPy, Pandas, Torch) are imported for data manipulation and building the AutoEncoder neural network. |
| Importing the Dataset                                 | The MovieLens datasets (movies, users, ratings) are imported using Pandas, but these datasets won't be used in this specific code. |
| Preparing the Training Set and Test Set               | The training and test sets are loaded from the files "u1.base" and "u1.test" in the ml-100k dataset, respectively. |
| Getting the Number of Users and Movies                | The total number of users and movies are calculated to determine the dimensions of the user-movie interaction matrix. |
| Converting the Data into an Array with Users in Lines and Movies in Columns | The training and test sets are converted into a list of lists where each list corresponds to a user's ratings for all movies. |
| Converting the Data into Torch Tensors                | The user-movie interaction matrices (training and test sets) are converted into Torch tensors, which are used in PyTorch for building the AutoEncoder neural network. |
| Creating the Architecture of the Neural Network (AutoEncoder) | A class SAE is defined, which represents the AutoEncoder. It consists of fully connected layers to encode and decode the user-movie interaction matrix. |
| Training the AutoEncoder                              | The AutoEncoder model is trained using Mean Squared Error (MSE) loss and RMSprop optimizer on the training set. |
| Testing the AutoEncoder                               | The trained AutoEncoder model is tested using the test set to evaluate its performance. |


In [ ]:
# Downloading the dataset
!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip
!ls

!wget "http://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip ml-1m.zip
!ls

# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

# Importing the dataset
# We won't be using this dataset.
movies = pd.read_csv('ml-1m/movies.dat', sep='::', header=None, engine='python', encoding='latin-1')
users = pd.read_csv('ml-1m/users.dat', sep='::', header=None, engine='python', encoding='latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header=None, engine='python', encoding='latin-1')

# Preparing the training set and the test set
training_set = pd.read_csv('ml-100k/u1.base', delimiter='\t')
training_set = np.array(training_set, dtype='int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter='\t')
test_set = np.array(test_set, dtype='int')

# Getting the number of users and movies
nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))

# Converting the data into an array with users in lines and movies in columns
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:, 1][data[:, 0] == id_users]
        id_ratings = data[:, 2][data[:, 0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

training_set = convert(training_set)
test_set = convert(test_set)

# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

# Creating the architecture of the Neural Network (AutoEncoder)
class SAE(nn.Module):
    def __init__(self):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr=0.01, weight_decay=0.5)

# Training the SAE
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.requires_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data * mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: ' + str(epoch) + ' loss: ' + str(train_loss / s))

# Testing the SAE
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.requires_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data * mean_corrector)
        s += 1.
print('test loss: ' + str(test_loss / s))
